In [1]:
# ----------------------------
# 📘 RNN Next Word Prediction
# ----------------------------
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import numpy as np

In [2]:
# ------------------------------------------
# Step 1: Input paragraph (training text)
# ------------------------------------------
text = """
I love learning about deep learning and natural language processing.
Deep learning models like RNN and LSTM understand sequential data.
I enjoy exploring how RNNs predict the next word in a sentence.
"""

In [9]:
# ------------------------------------------
# Step 2: Tokenization
# ------------------------------------------
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1  # +1 for padding
total_words

28

In [10]:
tokenizer.word_index

{'learning': 1,
 'i': 2,
 'deep': 3,
 'and': 4,
 'love': 5,
 'about': 6,
 'natural': 7,
 'language': 8,
 'processing': 9,
 'models': 10,
 'like': 11,
 'rnn': 12,
 'lstm': 13,
 'understand': 14,
 'sequential': 15,
 'data': 16,
 'enjoy': 17,
 'exploring': 18,
 'how': 19,
 'rnns': 20,
 'predict': 21,
 'the': 22,
 'next': 23,
 'word': 24,
 'in': 25,
 'a': 26,
 'sentence': 27}

In [11]:
# Create sequences of words (for training)
input_sequences = []
for line in text.split('.'):
    tokens = tokenizer.texts_to_sequences([line])[0]
    for i in range(2, len(tokens) + 1):
        seq = tokens[:i]
        input_sequences.append(seq)

In [12]:
input_sequences

[[2, 5],
 [2, 5, 1],
 [2, 5, 1, 6],
 [2, 5, 1, 6, 3],
 [2, 5, 1, 6, 3, 1],
 [2, 5, 1, 6, 3, 1, 4],
 [2, 5, 1, 6, 3, 1, 4, 7],
 [2, 5, 1, 6, 3, 1, 4, 7, 8],
 [2, 5, 1, 6, 3, 1, 4, 7, 8, 9],
 [3, 1],
 [3, 1, 10],
 [3, 1, 10, 11],
 [3, 1, 10, 11, 12],
 [3, 1, 10, 11, 12, 4],
 [3, 1, 10, 11, 12, 4, 13],
 [3, 1, 10, 11, 12, 4, 13, 14],
 [3, 1, 10, 11, 12, 4, 13, 14, 15],
 [3, 1, 10, 11, 12, 4, 13, 14, 15, 16],
 [2, 17],
 [2, 17, 18],
 [2, 17, 18, 19],
 [2, 17, 18, 19, 20],
 [2, 17, 18, 19, 20, 21],
 [2, 17, 18, 19, 20, 21, 22],
 [2, 17, 18, 19, 20, 21, 22, 23],
 [2, 17, 18, 19, 20, 21, 22, 23, 24],
 [2, 17, 18, 19, 20, 21, 22, 23, 24, 25],
 [2, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
 [2, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]]

In [13]:
# Pad all sequences to equal length
max_seq_len = max(len(seq) for seq in input_sequences)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

In [14]:
input_sequences

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  5],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  5,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  2,  5,  1,  6],
       [ 0,  0,  0,  0,  0,  0,  0,  2,  5,  1,  6,  3],
       [ 0,  0,  0,  0,  0,  0,  2,  5,  1,  6,  3,  1],
       [ 0,  0,  0,  0,  0,  2,  5,  1,  6,  3,  1,  4],
       [ 0,  0,  0,  0,  2,  5,  1,  6,  3,  1,  4,  7],
       [ 0,  0,  0,  2,  5,  1,  6,  3,  1,  4,  7,  8],
       [ 0,  0,  2,  5,  1,  6,  3,  1,  4,  7,  8,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  1, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  3,  1, 10, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  3,  1, 10, 11, 12],
       [ 0,  0,  0,  0,  0,  0,  3,  1, 10, 11, 12,  4],
       [ 0,  0,  0,  0,  0,  3,  1, 10, 11, 12,  4, 13],
       [ 0,  0,  0,  0,  3,  1, 10, 11, 12,  4, 13, 14],
       [ 0,  0,  0,  3,  1, 10, 11, 12,  4, 13, 14, 15],
       [ 0,  0,  3,  1, 10, 11,

In [15]:
# Split predictors and label
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [16]:
X

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  5],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  2,  5,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  2,  5,  1,  6],
       [ 0,  0,  0,  0,  0,  0,  2,  5,  1,  6,  3],
       [ 0,  0,  0,  0,  0,  2,  5,  1,  6,  3,  1],
       [ 0,  0,  0,  0,  2,  5,  1,  6,  3,  1,  4],
       [ 0,  0,  0,  2,  5,  1,  6,  3,  1,  4,  7],
       [ 0,  0,  2,  5,  1,  6,  3,  1,  4,  7,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  3,  1, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  3,  1, 10, 11],
       [ 0,  0,  0,  0,  0,  0,  3,  1, 10, 11, 12],
       [ 0,  0,  0,  0,  0,  3,  1, 10, 11, 12,  4],
       [ 0,  0,  0,  0,  3,  1, 10, 11, 12,  4, 13],
       [ 0,  0,  0,  3,  1, 10, 11, 12,  4, 13, 14],
       [ 0,  0,  3,  1, 10, 11, 12,  4, 13, 14, 15],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0

In [17]:
y

array([ 5,  1,  6,  3,  1,  4,  7,  8,  9,  1, 10, 11, 12,  4, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=int32)

In [18]:
# One-hot encode labels
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_words)

In [19]:
y

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

In [20]:
# ------------------------------------------
# Step 3: Define RNN Model
# ------------------------------------------
model = Sequential([
    Embedding(total_words, 64, input_length=max_seq_len - 1),
    SimpleRNN(128),
    Dense(total_words, activation='softmax')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
# ------------------------------------------
# Step 4: Train the model
# ------------------------------------------
model.fit(X, y, epochs=300, verbose=1)

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0345 - loss: 3.3545
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 0.2414 - loss: 3.2879
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.2414 - loss: 3.2229
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.3103 - loss: 3.1561
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.2414 - loss: 3.0868
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.2414 - loss: 3.0170
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.2414 - loss: 2.9483
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.2069 - loss: 2.8759
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.3448 - loss: 2.7938
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.3448 - loss: 2.7052
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3793 - loss: 2.6163
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4483 -

In [23]:
# ------------------------------------------
# Step 5: Predict next word
# ------------------------------------------
def predict_next_word(seed_text, next_words=1):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0))

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [24]:
# ------------------------------------------
# Step 6: Test the model
# ------------------------------------------
print(predict_next_word("I love learning", next_words=3))

I love learning about deep learning


In [31]:
print(predict_next_word("I enjoy exploring how RNNs predict the", next_words=5))

I enjoy exploring how RNNs predict the next word in a sentence


In [36]:
print(predict_next_word("what is lstm", next_words=5))

what is lstm learning learning like rnn and


In [ ]:



# I love learning about deep learning and natural language processing.
# Deep learning models like RNN and LSTM understand sequential data.
# I enjoy exploring how RNNs predict the next word in a sentence.